In [1]:
''' Reference : https://gist.github.com/jcanizalez/e089e3ab8eaf119f8ee622cfa364ed8c '''
# !pip install langchain-openai
!pip install wikipedia


[notice] A new release of pip is available: 23.2.1 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
import os
from dotenv import load_dotenv
from typing import Optional, Type
from pydantic import BaseModel
from langchain.tools import BaseTool

load_dotenv()

C:\Users\euiyoung.hwang\Git_Workspace\Vector_DB_with_LLM\.venv\Lib\site-packages\requests\__init__.py:102: RequestsDependencyWarning: urllib3 (1.26.20) or chardet (5.2.0)/charset_normalizer (2.0.12) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({})/charset_normalizer ({}) doesn't match a supported "


True

In [3]:
os.environ['CURL_CA_BUNDLE'] = ''

In [4]:
from langchain.tools import BaseTool
from typing import Optional
from pydantic import BaseModel, Field
from langchain_openai.chat_models import ChatOpenAI
from langchain.agents import initialize_agent
from langchain.llms import HuggingFaceHub
from langchain_huggingface import ChatHuggingFace, HuggingFacePipeline
import requests

from langchain.agents import (
    AgentType, initialize_agent, load_tools
)
import warnings
warnings.filterwarnings("ignore")

In [5]:
class PrometheusQueryToolConfig(BaseModel):
    prometheus_url: str = Field(default="http://localhost:9090")

In [6]:
class CustomToolInput(BaseModel):
    # query: str = Field()
    query: Optional[str] = "test"


In [7]:
class PrometheusQueryTool(BaseTool):
    # name = "Prometheus Query"
    # description = "Tool for querying a Prometheus server"
    # config: Optional[PrometheusQueryToolConfig] = None
    # args_schema: Type[BaseModel] = CustomToolInput
    
    def __init__(self, prometheus_url: str = "http://localhost:9090"):
        # super().__init__()
        # self.config = PrometheusQueryToolConfig(prometheus_url=prometheus_url)
        # self.args_schema = {}
        # self.args_schema: Type[BaseModel] = CustomToolInput
        pass

    def _run(self, query: str):
        params = {'query': query}
        response = requests.get(f"{self.prometheus_url}/query", params=params)
        print(response.status_code)
        if response.status_code == 200:
            return response.json()
        else:
            return f"Error: {response.text}"

    # def _arun(self, query: str):
    #     raise NotImplementedError("This tool does not support async")

In [8]:
# Initialize the Prometheus Query Tool with the URL of your Prometheus server
prometheus_tool = PrometheusQueryTool(prometheus_url="http://localhost:9090")

In [9]:

# Initialize LLM (ChatOpenAI)
# llm = ChatOpenAI(temperature=0, model_name='gpt-4')

# model_id='mistralai/Mistral-7B-v0.1'
model_id='HuggingFaceH4/zephyr-7b-beta'
llm=HuggingFaceHub(repo_id=model_id, model_kwargs={"temperature": 0.2, "max_length": 1280})
llm = ChatHuggingFace(llm=llm)

# Include the Prometheus tool in the list of tools
# tools = [prometheus_tool]


In [10]:
from langchain_core.prompts import PromptTemplate
from langchain import LLMChain

# template = """Answer in Korean.
template = """
질문: {question}
답변: """
# 프롬프트 템플릿 생성
prompt = PromptTemplate.from_template(template)

# LLM Chain 객체 생성
llm_chain = LLMChain(prompt=prompt, llm=llm)
# question = "what is the capital of korea?"
question = "what is elasticsearch"
# print(llm_chain.run(question=question))
# response = llm_chain.run(question=question)
# print('\nresponse\n')
# print(response)
response = llm_chain.invoke(question)
'''
{'question': 'what is the capital of South Korea?', 'text': '<|user|>\n질문: what is the capital of korea?\n답변: </s>\n<|assistant|>\nThe capital of South Korea is Seoul, and the capital of North Korea is Pyongyang. However, in this context, if the question is referring to the entire Korean Peninsula, then there is no single capital as it is divided into two separate countries.'}
'''
print(response['text'].split('<|assistant|>')[1].strip())



Elasticsearch is a popular open-source search and analytics engine built on top of the Lucene library. It provides a distributed, RESTful search and analytics capability for unstructured and semi-structured data. Elasticsearch is designed to be scalable, resilient, and flexible, making it a popular choice for powering search and analytics use cases in various industries, such as e-commerce, logistics, finance, and healthcare. It offers features such as real-time index


In [11]:

# Prompt for the agent
prompt = """
Use metrics to answer this question:
{text}
"""

# Initialize agent with tools
# agent = initialize_agent(
#     tools=tools,
#     llm=llm,
#     verbose=True,
# )

# agent = initialize_agent(
#     tools=tools, llm=llm, agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION, verbose=True
# )

In [12]:
# Example query to the agent
# response = agent(prompt.format(text="Can you check the status for all the pods in default namespace?"))
# print(response)